In [16]:
import json
import bs4 as bs
import requests
import pandas as pd

In [33]:
url = f'https://wiki.oceannetworks.ca/display/O2A/Sample+Code'
response = requests.get(url)

if (response.ok):
    # requestInfo = response._content #json.loads(str(response._content,'utf-8')) # convert the json response to an object
    requestInfo =response._content #json.loads(str(response._content,'utf-8')) # convert the json response to an object
else:
    if(response.status_code == 400):
        error = json.loads(str(response._content,'utf-8'))
        print(error) # json response contains a list of errors, with an errorMessage and parameter
    else:
        print ('Error {} - {}'.format(response.status_code,response.reason))

In [39]:
soup = bs.BeautifulSoup(requestInfo, 'lxml')
request_hrfs = soup.select("a[href*='display/O2A/Request']")
discover_hrfs = soup.select("a[href*='display/O2A/Discover']")

discover_urls = [ 'https://wiki.oceannetworks.ca' + hrf.get('href') for hrf in discover_hrfs]
request_urls = [ 'https://wiki.oceannetworks.ca' + hrf.get('href') for hrf in request_hrfs]


In [44]:
all_discovery_codes = []
for url in discover_urls:
    response1 = requests.get(url)

    if (response1.ok):
        requestInfo1 = response1._content #json.loads(str(response._content,'utf-8')) # convert the json response to an object
        soup1 = bs.BeautifulSoup(requestInfo1, 'lxml')
        contents = soup1.find('div', class_="wiki-content", id="main-content").text
        all_discovery_codes.append(contents)
    else:
        if(response1.status_code == 400):
            error = response1._content
            print(error) # json response contains a list of errors, with an errorMessage and parameter
        else:
            print ('Error {} - {}'.format(response1.status_code,response1.reason))


In [48]:
all_request_codes = []
for url in request_urls:
    response1 = requests.get(url)

    if (response1.ok):
        requestInfo1 = response1._content #json.loads(str(response._content,'utf-8')) # convert the json response to an object
        soup1 = bs.BeautifulSoup(requestInfo1, 'html.parser')
        contents = soup1.find('div', class_="wiki-content", id="main-content").text
        all_request_codes.append(contents)
    else:
        if(response1.status_code == 400):
            error = response1._content
            print(error) # json response contains a list of errors, with an errorMessage and parameter
        else:
            print ('Error {} - {}'.format(response1.status_code,response1.reason))


In [53]:
with open('discover_codes.txt', 'w') as f:
    for item in all_discovery_codes:
        f.write("%s\n" % item)

In [54]:
with open('request_codes.txt', 'w') as f:
    for item in all_request_codes:
        f.write("%s\n" % item)